In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pyarrow.parquet as pq
import gc
import pywt
from statsmodels.robust import mad
import scipy
from scipy import signal
from scipy.signal import butter
from tqdm import tqdm_notebook
import os
from multiprocessing import Pool

from numba import jit

import warnings

# Suppress pandas future warnings, I am using different library versions locally
# that do not raise warnings.
warnings.simplefilter(action='ignore', category=FutureWarning)

data_dir = '../input'

In [2]:
print(scipy.__version__)

1.1.0


In [3]:
# 800,000 data points taken over 20 ms
# Grid operates at 50hz, 0.02 * 50 = 1, so 800k samples in 20 milliseconds will capture one complete cycle
n_samples = 800000

# Sample duration is 20 miliseconds
sample_duration = 0.02

# Sample rate is the number of samples in one second
# Sample rate will be 40mhz
sample_rate = n_samples * (1 / sample_duration)

In [4]:
def maddest(d, axis=None):
    """
    Mean Absolute Deviation
    """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [5]:
def high_pass_filter(x, low_cutoff=1000, sample_rate=sample_rate):
    """
    From @randxie https://github.com/randxie/Kaggle-VSB-Baseline/blob/master/src/utils/util_signal.py
    Modified to work with scipy version 1.1.0 which does not have the fs parameter
    """
    
    # nyquist frequency is half the sample rate https://en.wikipedia.org/wiki/Nyquist_frequency
    nyquist = 0.5 * sample_rate
    norm_low_cutoff = low_cutoff / nyquist
    
    # Fault pattern usually exists in high frequency band. According to literature, the pattern is visible above 10^4 Hz.
    # scipy version 1.2.0
    #sos = butter(10, low_freq, btype='hp', fs=sample_fs, output='sos')
    
    # scipy version 1.1.0
    sos = butter(10, Wn=[norm_low_cutoff], btype='highpass', output='sos')
    filtered_sig = signal.sosfilt(sos, x)

    return filtered_sig

In [6]:
def denoise_signal( x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec( x, wavelet, mode="per", level=level)
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest( coeff[-level] )

    # Calculte the univeral threshold
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode='hard' ) for i in coeff[1:] )
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec( coeff[1:], wavelet, mode='per' )

In [7]:
def denoise_signal_2( x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec( x, wavelet, mode="per", level=level)
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest( coeff[-level] )

    # Calculte the univeral threshold
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode='hard' ) for i in coeff[1:] )
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec( coeff[0:], wavelet, mode='per' )

In [8]:
def remove_corona(x_dn):
    index = pd.Series(x_dn).loc[np.abs(x_dn)>0].index
    corona_idx = []
    for idx in index:
        for i in range(1,maxDistance+1):
            if idx+i < pd.Series(x_dn).shape[0]:
                if x_dn[idx+i]/(x_dn[idx]+1e-04)<-maxHeightRatio:
                    x_dn[idx:idx+maxTicksRemoval] = 0
                    corona_idx.append(idx)
    return x_dn, corona_idx

In [9]:
maxDistance = 10
maxHeightRatio = 0.25
maxTicksRemoval =500

In [10]:
subset_train = pq.read_pandas(data_dir + '/train.parquet').to_pandas()

In [11]:
stdamp_id = np.zeros(subset_train.shape[1])
stdamppos_id = np.zeros(subset_train.shape[1])
stdampneg_id = np.zeros(subset_train.shape[1])

meanamppos_id = np.zeros(subset_train.shape[1])
meanampneg_id = np.zeros(subset_train.shape[1])

numpeaksq2_id = np.zeros(subset_train.shape[1])
numpospeaksq2_id = np.zeros(subset_train.shape[1])
numnegpeaksq2_id = np.zeros(subset_train.shape[1])

numpeaksq3_id = np.zeros(subset_train.shape[1])
numpospeaksq3_id = np.zeros(subset_train.shape[1])
numnegpeaksq3_id = np.zeros(subset_train.shape[1])

numpeaksq4_id = np.zeros(subset_train.shape[1])
numpospeaksq4_id = np.zeros(subset_train.shape[1])
numnegpeaksq4_id = np.zeros(subset_train.shape[1])

coefampq2q3_id = np.zeros(subset_train.shape[1])
coefampq2q4_id = np.zeros(subset_train.shape[1])

stdampq2_id = np.zeros(subset_train.shape[1])
stdposampq2_id = np.zeros(subset_train.shape[1])
stdnegampq2_id = np.zeros(subset_train.shape[1])

stdampq3_id = np.zeros(subset_train.shape[1])
stdposampq3_id = np.zeros(subset_train.shape[1])
stdnegampq3_id = np.zeros(subset_train.shape[1])

stdampq4_id = np.zeros(subset_train.shape[1])
stdposampq4_id = np.zeros(subset_train.shape[1])
stdnegampq4_id = np.zeros(subset_train.shape[1])

meanampq2_id = np.zeros(subset_train.shape[1])
meanposampq2_id = np.zeros(subset_train.shape[1])
meannegampq2_id = np.zeros(subset_train.shape[1])

meanampq3_id = np.zeros(subset_train.shape[1])
meanposampq3_id = np.zeros(subset_train.shape[1])
meannegampq3_id = np.zeros(subset_train.shape[1])

meanampq4_id = np.zeros(subset_train.shape[1])
meanposampq4_id = np.zeros(subset_train.shape[1])
meannegampq4_id = np.zeros(subset_train.shape[1])


In [12]:
for col in tqdm_notebook(range(0, subset_train.shape[1], 3)):
    X_HP = []
    X_DN = []
    
    X_HP.append(high_pass_filter(subset_train.iloc[:,col], low_cutoff=10000, sample_rate=sample_rate))
    X_HP.append(high_pass_filter(subset_train.iloc[:,col+1], low_cutoff=10000, sample_rate=sample_rate))
    X_HP.append(high_pass_filter(subset_train.iloc[:,col+2], low_cutoff=10000, sample_rate=sample_rate))
    
    X_DN.append(denoise_signal(X_HP[0], wavelet='haar', level=1))
    X_DN.append(denoise_signal(X_HP[1], wavelet='haar', level=1))
    X_DN.append(denoise_signal(X_HP[2], wavelet='haar', level=1))
    
    X_DN[0], corona_idx = remove_corona(X_DN[0])
    X_DN[1], corona_idx = remove_corona(X_DN[1])
    X_DN[2], corona_idx = remove_corona(X_DN[2])
    
    total = X_DN[0] + X_DN[1] + X_DN[2]
    total = pd.Series(total)
      
    stdamp_id[col:col+3] = np.std(total)
    stdamppos_id[col:col+3] = np.std(total[total<0])
    stdampneg_id[col:col+3] = np.std(total[total>0])

    meanamppos_id[col:col+3] = np.mean(total[total<0])
    meanampneg_id[col:col+3] = np.mean(total[total>0])
    
    numpeaksq2_id[col:col+3] = total[100000:200000].loc[total!=0].count()
    numpospeaksq2_id[col:col+3] = total[100000:200000].loc[total>0].count()
    numnegpeaksq2_id[col:col+3] = total[100000:200000].loc[total<0].count()
    
    numpeaksq3_id[col:col+3] = total[200000:300000].loc[total!=0].count()
    numpospeaksq3_id[col:col+3] = total[200000:300000].loc[total>0].count()
    numnegpeaksq3_id[col:col+3] = total[200000:300000].loc[total<0].count()
    
    numpeaksq4_id[col:col+3] = total[300000:400000].loc[total!=0].count()
    numpospeaksq4_id[col:col+3] = total[300000:400000].loc[total>0].count()
    numnegpeaksq4_id[col:col+3] = total[300000:400000].loc[total<0].count()
    
    coefampq2q3_id = np.corrcoef(total[100000:200000], total[200000:300000])[0][1]
    coefampq2q4_id = np.corrcoef(total[100000:200000], total[300000:400000])[0][1]
    
    stdampq2_id[col:col+3] = np.std(total[100000:200000].loc[total!=0])
    stdposampq2_id[col:col+3] = np.std(total[100000:200000].loc[total>0])
    stdnegampq2_id[col:col+3] = np.std(total[100000:200000].loc[total<0])
    
    stdampq3_id[col:col+3] = np.std(total[200000:300000].loc[total!=0])
    stdposampq3_id[col:col+3] = np.std(total[200000:300000].loc[total>0])
    stdnegampq3_id[col:col+3] = np.std(total[200000:300000].loc[total<0])
    
    stdampq4_id[col:col+3] = np.std(total[300000:400000].loc[total!=0])
    stdposampq4_id[col:col+3] = np.std(total[300000:400000].loc[total>0])
    stdnegampq4_id[col:col+3] = np.std(total[300000:400000].loc[total<0])
    
    meanampq2_id[col:col+3] = np.mean(total[100000:200000].loc[total!=0])
    meanposampq2_id[col:col+3] = np.mean(total[100000:200000].loc[total>0])
    meannegampq2_id[col:col+3] = np.mean(total[100000:200000].loc[total<0])
    
    meanampq3_id[col:col+3] = np.mean(total[200000:300000].loc[total!=0])
    meanposampq3_id[col:col+3] = np.mean(total[200000:300000].loc[total>0])
    meannegampq3_id[col:col+3] = np.mean(total[200000:300000].loc[total<0])
    
    meanampq4_id[col:col+3] = np.mean(total[300000:400000].loc[total!=0])
    meanposampq4_id[col:col+3] = np.mean(total[300000:400000].loc[total>0])
    meannegampq4_id[col:col+3] = np.mean(total[300000:400000].loc[total<0])

/home/cwademo123/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/cwademo123/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [13]:
np.save('../features/stdamp_id.npy', stdamp_id)
np.save('../features/stdamppos_id.npy', stdamppos_id)
np.save('../features/stdampneg_id.npy', stdampneg_id )

np.save('../features/meanamppos_id.npy', meanamppos_id)
np.save('../features/meanampneg_id.npy', meanampneg_id)

np.save('../features/numpeaksq2_id.npy', numpeaksq2_id)
np.save('../features/numpeaksq3_id.npy', numpeaksq3_id )
np.save('../features/numpeaksq4_id.npy', numpeaksq4_id)

np.save('../features/numpospeaksq2_id.npy', numpospeaksq2_id)
np.save('../features/numpospeaksq3_id.npy', numpospeaksq3_id )
np.save('../features/numpospeaksq4_id.npy', numpospeaksq4_id)

np.save('../features/numnegpeaksq2_id.npy', numnegpeaksq2_id)
np.save('../features/numnegpeaksq3_id.npy', numnegpeaksq3_id )
np.save('../features/numnegpeaksq4_id.npy', numnegpeaksq4_id)

np.save('../features/coefampq2q3_id.npy', coefampq2q3_id)
np.save('../features/coefampq2q4_id.npy', coefampq2q4_id)

np.save('../features/stdampq2_id.npy', stdampq2_id)
np.save('../features/stdampq3_id.npy', stdampq3_id )
np.save('../features/stdampq4_id.npy', stdampq4_id)

np.save('../features/stdposampq2_id.npy', stdposampq2_id)
np.save('../features/stdposampq3_id.npy', stdposampq3_id )
np.save('../features/stdposampq4_id.npy', stdposampq4_id)

np.save('../features/stdnegampq2_id.npy', stdnegampq2_id)
np.save('../features/stdnegampq3_id.npy', stdnegampq3_id )
np.save('../features/stdnegampq4_id.npy', stdnegampq4_id)

np.save('../features/meanampq2_id.npy', meanampq2_id)
np.save('../features/meanampq3_id.npy', meanampq3_id )
np.save('../features/meanampq4_id.npy', meanampq4_id)

np.save('../features/meanposampq2_id.npy', meanposampq2_id)
np.save('../features/meanposampq3_id.npy', meanposampq3_id )
np.save('../features/meanposampq4_id.npy', meanposampq4_id)

np.save('../features/meannegampq2_id.npy', meannegampq2_id)
np.save('../features/meannegampq3_id.npy', meannegampq3_id )
np.save('../features/meannegampq4_id.npy', meannegampq4_id)
